<a href="https://colab.research.google.com/github/hoa92ng/Homework/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import numpy as np
import torch.utils.data
from torch import nn
from torchvision import models
import sys, os
# print(os.getcwd())
sys.path.append('./src')
import patchcore
from patchcore import patchcore as patchcore_model
from siamese import SiameseNetwork
from torch.utils.data import TensorDataset, ConcatDataset, DataLoader
from torchvision.transforms import v2
from typing import Callable, Any
import glob, os
import random
from UNetwork import Resnet_AutoEncoder_Network, Resnet_Network

from torchvision.models import resnet50

class Custom_Dataset_All(Dataset):
    """
    URL = https://www.robots.ox.ac.uk/~vgg/data/pets/
    """
    def __init__(self,
                 image_folder: str = r"xxtion\mvtech_anomaly_detection",
                 image_name: str = "bottle",
                 split: str = "train",
                 transform: Callable = None,
                **kwargs):
        self._sub_dataset = ('bottle',  'cable',  'capsule',  'carpet',  'grid', 'hazelnut', 'leather',  'metal_nut',  'pill', 'screw', 'tile', 'toothbrush', 'transistor', 'wood', 'zipper')
        self.image_name = image_name
        self.img_folder = image_folder
        self.transforms = transform
        self.imgs_pos, self.imgs_neg = self.get_image_paths(img_folder=image_folder)

    def get_image_paths(self, img_folder, split: str = "train"):
        img_positive = []
        img_negative = []
        # Define the file extensions for images
        image_extensions = ('*.png', '*.jpg', '*.jpeg', '*.gif', '*.bmp', '*.tiff')
        # Initialize an empty list to store image paths
        for img_name in self._sub_dataset:
            for ext in image_extensions:
                img_fold = os.path.join(img_folder, img_name, "train", "good")
                if (img_name == self.image_name):
                    img_positive.extend(glob.glob(os.path.join(img_fold, ext)))
                else:
                    img_negative.extend(glob.glob(os.path.join(img_fold, ext)))

        img_positive = img_positive[:int(len(img_positive) * 0.75)] if split == "train" else img_positive[int(len(img_positive) * 0.75):]
        img_positive = [self.pil_loader(x) for x in img_positive]
        img_negative = [self.pil_loader(x) for x in img_negative]
        random.shuffle(img_positive)
        random.shuffle(img_negative)
        return img_positive, img_negative

    def __len__(self):
        return len(self.imgs_pos)

    def pil_loader(self, path: str) -> Image.Image:
        # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
        with open(path, "rb") as f:
            img = Image.open(f).convert("RGB")

            if self.transforms is not None:
                img = self.transforms(img)
            return img

    def default_loader(self, path: str) -> Any:
            return self.pil_loader(path)

    def __getitem__(self, idx):
        valid_indices = [i for i in range(len(self.imgs_pos)) if i != idx]
        random_index = random.choice(valid_indices)
        img_anchor = self.imgs_pos[idx]
        img_pos = self.imgs_pos[random_index]
        img_neg = self.imgs_neg[random.randint(0, len(self.imgs_neg) - 1)]
        return img_anchor, img_pos,  img_neg

def _standard_patchcore(image_dimension):
    patchcore_instance = patchcore_model.PatchCore(torch.device("cpu"))
    backbone = models.wide_resnet50_2(pretrained=False)
    backbone.name, backbone.seed = "wideresnet50", 0
    patchcore_instance.load(
        backbone=backbone,
        layers_to_extract_from=["layer2", "layer3"],
        device=torch.device("cpu"),
        input_shape=[3, image_dimension, image_dimension],
        pretrain_embed_dimension=1024,
        target_embed_dimension=1024,
        patchsize=3,
        patchstride=1,
        spade_nn=2,
        sub_model=None
    )
    return patchcore_instance

if __name__ == "__main__":
    epoch_num = 100
    batch_size = 8

    network_predict = Resnet_Network()
    network_autoencoder = Resnet_AutoEncoder_Network()
    criterion = nn.BCEWithLogitsLoss()
    criterion_2 = nn.MSELoss()

    optimizer_predict = torch.optim.Adam(network_predict.parameters(), lr=0.001)
    optimizer_autoencoder = torch.optim.Adam(network_autoencoder.parameters(), lr=0.001)

    transfomation = v2.Compose([
            v2.Resize(256),
            v2.CenterCrop(224),
            v2.ToTensor(),
            # transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),

        ])
    _dataset = Custom_Dataset_All(transform=transfomation)
    _data_loader = DataLoader(_dataset, batch_size=batch_size, shuffle=True)

    _dataset_test = Custom_Dataset_All(transform=transfomation, split="test")
    _test_data_loader = DataLoader(_dataset_test, batch_size=batch_size, shuffle=True)
    # model = _standard_patchcore(224)
    for epoch in range(epoch_num):
        print(f"=========== epoch {epoch} ===========")
        loss_item = 0.
        valid_loss = 0.
        save_loss = 999999999999
        network_autoencoder.train()
        network_predict.train()
        for i, image in enumerate(_data_loader):
            if (image[0].shape[0] < batch_size): continue
            img0, img1 = image[0], image[1]
            img0, img1 = img0.cuda(), img1.cuda()

            lbl0 = torch.zeros((img0.shape[0], 1))
            lbl1 = torch.ones((img0.shape[0], 1))
            lbl = torch.concat((lbl0, lbl1), dim=0).to(device='cuda')

            optimizer_autoencoder.zero_grad()
            optimizer_predict.zero_grad()

            x_mask, out_noise = network_autoencoder(img0)
            l2 = criterion_2(x_mask, out_noise)
            l2.backward()
            optimizer_autoencoder.step()

            out_pre = network_predict(img0, img1)
            l1 = criterion(out_pre, lbl)
            l1.backward()
            optimizer_predict.step()

            loss_item += l1.item()
            if i % 1 == 0:
                print(f"Epoch [{epoch+1}/{epoch_num}], Step [{i}/{len(_data_loader)}], Loss: {(loss_item/(i+1)):.4f}, l1: {l1.item()}, l2: {l2.item()}")


    #     network.eval()
    #     with torch.no_grad():
    #         for i, image in enumerate(_test_data_loader):
    #             if (image[0].shape[0] < 2): continue
    #             img0, img1 = image[0], image[1]
    #             img0, img1 = img0.cuda(), img1.cuda()

    #             lbl0 = torch.zeros((img0.shape[0], 1))
    #             lbl1 = torch.ones((img0.shape[0], 1))
    #             lbl = torch.concat((lbl0, lbl1), dim=0).to(device='cuda')

    #             out_pre, x_mask, out_noise = network(img0, img1)
    #             loss = criterion(out_pre, lbl) + criterion_2(x_mask, out_noise)
    #             valid_loss += loss.item()
    #             if valid_loss < save_loss:
    #                 torch.save(network.state_dict(), f'./sub_model/siamese_wide_resnet50_2_best_loss_{valid_loss}.pth')
    #                 save_loss = valid_loss
    #         print(f"Epoch [{epoch+1}/{epoch_num}], Valid Loss: {(valid_loss/(len(_data_loader)+1)):.4f}")


    # torch.save(network.state_dict(), f'./sub_model/siamese_wide_resnet50_2_final.pth')

In [ ]:
import torch
from UNetwork import Unknown_Network
from torchvision.transforms import v2
from PIL import Image

import os
from glob import glob

def get_image_paths(img_folder, split: str = "test", transform=None):
    images = []
    # Define the file extensions for images
    image_extensions = ('*.png', '*.jpg', '*.jpeg', '*.gif', '*.bmp', '*.tiff')
    # Initialize an empty list to store image paths
    seach_folder = os.path.join(img_folder, split)
    folder_names =[os.path.join(seach_folder, x) for x in os.listdir(seach_folder)]
    for folder in folder_names:
        for ext in image_extensions:
            for path in glob(os.path.join(folder, ext)):
                with open(path, "rb") as f:
                    img = Image.open(f).convert("RGB")
                    if transform is not None:
                        img = transform(img)
                    images.append([os.path.basename(folder), img, path])
                    print(path)
    return images

if __name__ == "__main__":

    image_path = r"Dxxxn\mvtech_anomaly_detection\bottle"
    model_path = './sub_model/siamese_wide_resnet50_2_final.pth'

    transfomation = v2.Compose([
            v2.Resize(256),
            v2.CenterCrop(224),
            v2.ToTensor(),
        ])

    network = Unknown_Network().to(device='cuda')
    network.load_state_dict(torch.load(model_path))

    imgs = get_image_paths(image_path, transform=transfomation)
    network.eval()
    with torch.no_grad():
        loss_item = 0.
        valid_loss = 0.
        save_loss = 999999999999
        network.train()
        for i, image in enumerate(imgs):
            # img_noise = torch.clip(image[1] +  torch.tensor(0).expand(*image[1].size()).float().normal_() * 0.5 + 0, 0, 1)
            img0 = torch.unsqueeze(image[1], 0).cuda()
            out_pre, x_mask, _ = network(img0, img0, with_noise=True)
            print(image[0], torch.sigmoid(out_pre).tolist(), image[2], '\n')

In [ ]:
import torch
from torch import nn
from torchvision.models import resnet18
from resnet_autoencoder_arc import ResNetAutoEncoder, get_configs
from torchvision.transforms import v2

class GaussianNoise(nn.Module):
    """Gaussian noise regularizer.

    Args:
        sigma (float, optional): relative standard deviation used to generate the
            noise. Relative means that it will be multiplied by the magnitude of
            the value your are adding the noise to. This means that sigma can be
            the same regardless of the scale of the vector.
        is_relative_detach (bool, optional): whether to detach the variable before
            computing the scale of the noise. If `False` then the scale of the noise
            won't be seen as a constant but something to optimize: this will bias the
            network to generate vectors with smaller values.
    """
    def __init__(self, sigma=0.1, is_relative_detach=True, device='cuda'):
        super().__init__()
        # self.sigma = sigma
        self.sigma = nn.Parameter(torch.rand(1).to(device=device))
        self.is_relative_detach = is_relative_detach
        self.register_buffer('noise', torch.tensor(0).to(device=device))
        self.blur = v2.GaussianBlur(3)
        # self.noise = torch.tensor(0).to(device)

    def forward(self, x, with_noise=False):
        if self.sigma != 0:
        # if self.training != 0:
            # self.sigma = torch.clip(self.sigma, 0, 1)
            scale = self.sigma * x.detach() if self.is_relative_detach else self.sigma * x
            sampled_noise = self.noise.expand(*x.size()).float().normal_() * scale
            x_noise = x + sampled_noise
            torch.clamp(x_noise, 0, 1)
            x_noise = self.blur(x_noise)
            if with_noise:
                return x_noise, x_noise - x
        return x

class Unknown_Network(nn.Module):
    def __init__(self, device='cuda'):
        super(Unknown_Network, self).__init__()

        self.noise = GaussianNoise(is_relative_detach=False)

        configs, bottleneck = get_configs("resnet18")
        self.mask_generator = ResNetAutoEncoder(configs, bottleneck).to(device=device)

        self.predictor = resnet18()
        self.predictor.fc = nn.LazyLinear(out_features=1)
        self.predictor = self.predictor.to(device=device)



    def forward(self, x, x_pos, with_noise=True):
        if with_noise:
            x_noise, noise = self.noise(x, with_noise=with_noise)
            x_mask = self.mask_generator(x_noise)
            x_neg = torch.clip(x + x_mask, 0, 1)
            x_pre = torch.concat((x_neg, x_pos), dim=0) if x_pos is not None else x
            out_pre = self.predictor(x_pre)
            return out_pre, x_mask, noise
        else:
            x_noise = x
            x_mask = self.mask_generator(x_noise)
            x_pre = torch.concat((x_noise, x_pos), dim=0) if x_pos is not None else x
            out_pre = self.predictor(x_pre)
            return out_pre, x_mask

class Resnet_AutoEncoder_Network(nn.Module):
    def __init__(self, device='cuda'):
        super(Resnet_AutoEncoder_Network, self).__init__()
        self.noise = GaussianNoise(is_relative_detach=False)
        configs, bottleneck = get_configs("resnet18")
        self.mask_generator = ResNetAutoEncoder(configs, bottleneck).to(device=device)

    def forward(self, x, with_noise=True):
        if with_noise:
            x_noise, noise = self.noise(x, with_noise=with_noise)
            x_mask = self.mask_generator(x_noise)
            return x_mask, noise
        else:
            x_mask = self.mask_generator(x)
            return x_mask


class Resnet_Network(nn.Module):
    def __init__(self, device='cuda'):
        super(Resnet_Network, self).__init__()

        self.predictor = resnet18()
        self.predictor.fc = nn.LazyLinear(out_features=1)
        self.predictor = self.predictor.to(device=device)

    def forward(self, x, x_pos):
        x_pre = torch.concat((x, x_pos), dim=0) if x_pos is not None else x
        out_pre = self.predictor(x_pre)
        return out_pre

